In [161]:
# %% 
"""
Note: I need to import the data from parameters.py
"""

import pyomo.environ as pyo
import numpy as np


import parameters
from parameters import *

from analysis import plot_solution, plot_solution_with_map, plot_solution_with_covering_radius, plot_solution_with_map_zoom, generate_facility_summary, display_selected_variables_CAC, plot_solution_with_model_dependent_covering_radius

# Change scenario dynamically
scenario_name = "pugnido_baseline"  # New scenario
parameters.scenario_name = scenario_name  # Update it in the session
importlib.reload(parameters)  # Reload the module to apply changes

# Display parameters for the new scenario
print(parameters.params)


{'location_file': 'individual_refugee_camps/pugnido_baseline.geojson', 'distance_matrix': 'individual_refugee_camps/distance_matrix_pugnido_baseline.xlsx', 'HFs_to_locate': [2, 1], 't1max': 500, 't2max': 3500, 'workers_to_allocate': [2, 18, 18], 'total_population': 700, 'demand_rate_opening_hours': [0.01428571429, 0.00428571428, 0.00142857142], 'demand_rate_closing_hours': [0.00428571428, 0.00285714285, 0.00142857142], 'working_hours': [7, 8, 8], 'service_time': [0.5, 1, 2], 'lb_workers': [[0, 1], [1, 2], [1, 2]], 'ub_workers': [[1, 2], [6, 8], [6, 8]], 'services_at_HFs': [[1, 0], [1, 1], [0, 1]], 'services_per_worker': [[1, 0, 1], [1, 1, 0], [0, 1, 1]]}


In [130]:
def create_workforce_model(model_data, open_hps, open_hcs, open_facilities, first_assignment, second_assignment):
    
    m = pyo.ConcreteModel()

    # Function to remove all components from the model
    def remove_all_components(m):
        components = list(m.component_objects())
        for comp in components:
            m.del_component(comp)
    
    # Remove all components from the model
    remove_all_components(m)

    # Define sets
    m.I = pyo.Set(initialize = model_data['I'])  # demand points
    m.J = pyo.Set(initialize = open_facilities)  # Only open facilities
    m.P = pyo.Set(initialize = model_data['P'])  # types of professionals
    m.S = pyo.Set(initialize = model_data['S'])  # services
    m.L = pyo.Set(initialize = model_data['L'])  # types of facilities

    # Define parameters
    m.n_HF = pyo.Param(m.L, initialize = model_data['n_HF'], within = pyo.Integers)
    m.d1 = pyo.Param(m.I, m.S, initialize = model_data['d1'], within = pyo.NonNegativeReals)
    m.d2 = pyo.Param(m.I, m.S, initialize = model_data['d2'], within = pyo.NonNegativeReals)
    m.t = pyo.Param(m.I, m.J, initialize = {(i, j): model_data['t'].loc[i, j] for i in model_data['t'].index for j in m.J})
    m.n_W = pyo.Param(m.P, initialize = model_data['n_W'], within = pyo.Integers)
    m.lb = pyo.Param(m.P, m.L, initialize = model_data['lb'], within = pyo.Integers)
    m.ub = pyo.Param(m.P, m.L, initialize = model_data['ub'], within = pyo.Integers)
    m.a_HF = pyo.Param(m.S, m.L, initialize = model_data['a_HF'], within = pyo.Binary)
    m.a_W = pyo.Param(m.P, m.S, initialize = model_data['a_W'], within = pyo.Binary)
    m.q = pyo.Param(m.S, initialize = model_data['q'], within = pyo.NonNegativeReals)
    m.h = pyo.Param(m.P, initialize = model_data['h'], within = pyo.NonNegativeReals)

    # Define decision variables
    m.w = pyo.Var(m.J, m.P, within = pyo.NonNegativeIntegers)
    m.f1 = pyo.Var(m.I, m.J, m.S, within = pyo.NonNegativeIntegers)
    m.f2 = pyo.Var(m.I, m.J, m.S, within = pyo.NonNegativeIntegers)

    # Define the objective
    m.obj = pyo.Objective(
        expr=pyo.quicksum(m.f1[i, j, s] + m.f2[i, j, s] for i in m.I for j in m.J for s in m.S),
        sense=pyo.maximize
    )

    # Constraint definitions (e.g., demand satisfaction, worker capacity, etc.)

    
    @m.Constraint(m.I, m.J, m.S)
    def R15_relation_flow_first_assignment(m, i, j, s):
        return m.f1[i, j, s] <= m.d1[i, s] * (1 if first_assignment[i] == j else 0)
    

    @m.Constraint(m.I, m.J, m.S)
    def R16_relation_flow_open_facility(m, i, j, s):
        if j in open_hps:
            return m.f1[i, j, s] <= m.d1[i, s] * m.a_HF[s, 'hp']
        elif j in open_hcs:
            return m.f1[i, j, s] <= m.d1[i, s] * m.a_HF[s, 'hc']
        return pyo.Constraint.Skip
    

    @m.Constraint(m.I, m.J, m.S)
    def R17_relation_flow_second_assignment(m, i, j, s):
        return m.f2[i, j, s] <= m.d2[i, s] * (1 if second_assignment[i] == j else 0)


    @m.Constraint(m.I, m.J, m.S)
    def R18_relation_flow_open_HC(m, i, j, s):
        if j in open_hcs:
            return m.f2[i, j, s] <= m.d2[i, s] * m.a_HF[s, 'hc']
        return pyo.Constraint.Skip


    @m.Constraint(m.J, m.S)
    def R19_satisfied_demand_HFs(m, j, s):
        return pyo.quicksum(m.f1[i,j,s] + m.f2[i,j,s] for i in m.I) <= (1/m.q[s]) * pyo.quicksum(m.h[p]*m.a_W[p,s]*m.w[j,p] for p in m.P)
    

    @m.Constraint(m.J)
    def R20_time_spent_demand_HFs(m, j):
        return pyo.quicksum(m.q[s] * (m.f1[i,j,s] + m.f2[i,j,s]) for i in m.I for s in m.S) <= pyo.quicksum(m.h[p]*m.w[j,p] for p in m.P)
      

    @m.Constraint(m.P)
    def R21_allocation_workers(m, p):
        return pyo.quicksum(m.w[j,p] for j in m.J) <= m.n_W[p]
    

    @m.Constraint(m.J, m.P)
    def R22_upper_bounds_workers(m, j, p):
        if j in open_hps:
            return m.w[j, p] <= m.ub[p, 'hp']
        elif j in open_hcs:
            return m.w[j, p] <= m.ub[p, 'hc']
        return pyo.Constraint.Skip


    @m.Constraint(m.J, m.P)
    def R23_lower_bounds_workers(m, j, p):
        if j in open_hps:
            return m.lb[p,'hp'] <= m.w[j,p]
        elif j in open_hcs:
            return m.lb[p,'hc'] <= m.w[j, p] 
        return pyo.Constraint.Skip
    

    return m


In [188]:
 open_hps_c 

array(['j2', 'j9'], dtype=object)

In [191]:
def fitness(model_data, open_hps, open_hcs):
    """
    Compute the fitness of a given facility configuration by assigning demand and solving an LP.
    
    Inputs:
        open_hcs (set): Set of open HC locations
        open_hps (set): Set of open HP locations
        model_data (dict): Dictionary containing all model parameters (same as used in Pyomo model)
    
    Returns:
        satisfied_demand (float): Total satisfied demand
        lost_demand (float): Total lost demand
        max_dist_first_assignment (float): Maximum distance in first assignment
        max_dist_second_assignment (float): Maximum distance in second assignment
    """

    # Step 1: Assign demand points to nearest open facilities

    first_assignment = {}  # i -> j (first assignment)
    second_assignment = {}  # i -> j (second assignment)
    max_dist_first_per_camp = {}
    max_dist_second_per_camp = {}

    open_facilities = np.concatenate([open_hps, open_hcs])

    for c in model_data['C']: # Iterate over each refugee camp
        I_c = model_data['I_c'][c] # Demand points in camp c
        J_c = np.array([j for j in open_facilities if j in model_data['J_c'][c]], dtype=object)  # Open facilities in camp c as an array

        if J_c.size == 0:  # If no facilities are open in this camp, skip assignment
            max_dist_first_per_camp[c] = 0
            max_dist_second_per_camp[c] = 0
            continue

        max_dist_first_per_camp[c] = 0
        max_dist_second_per_camp[c] = 0


        for i in I_c:  

            # First assignment: Closest HP or HC in the same camp
            open_hps_c = np.array([j for j in open_hps if j in J_c], dtype=object)  # Open HPs in camp c as an array

            if open_hps_c.size > 0:  # If there are open HCs in this camp
                closest_first = open_hps_c[np.argmin([model_data['t'].loc[i, j] for j in open_hps_c])]
                first_assignment[i] = closest_first
                max_dist_first_per_camp[c] = max(max_dist_first_per_camp[c], model_data['t'].loc[i, closest_first])
            else:
                first_assignment[i] = None  # No valid second assignment possible

            # Second assignment: Closest HC in the same camp
            open_hcs_c = np.array([j for j in open_hcs if j in J_c], dtype=object)  # Open HCs in camp c as an array

            if open_hcs_c.size > 0:  # If there are open HCs in this camp
                closest_second = open_hcs_c[np.argmin([model_data['t'].loc[i, j] for j in open_hcs_c])]
                second_assignment[i] = closest_second
                max_dist_second_per_camp[c] = max(max_dist_second_per_camp[c], model_data['t'].loc[i, closest_second])
            else:
                second_assignment[i] = None  # No valid second assignment possible

    # Compute the sum of the maximum distances across all camps
    sum_max_dist_first_assignment = sum(max_dist_first_per_camp.values())
    sum_max_dist_second_assignment = sum(max_dist_second_per_camp.values())

    # Step 2: Solve LP for Workforce Allocation

    m = create_workforce_model(model_data, open_hps, open_hcs, open_facilities, first_assignment, second_assignment)

    # Solve LP
    solver = pyo.SolverFactory('cplex')
    solver.solve(m)

    # Compute results
    satisfied_demand = pyo.value(m.obj)
    total_demand = sum(m.d1[i, s] + m.d2[i, s] for i in m.I for s in m.S)
    lost_demand = total_demand - satisfied_demand

    return satisfied_demand, lost_demand, sum_max_dist_first_assignment, sum_max_dist_second_assignment


In [192]:
# Open facilities (based on some external logic or decision)
open_hps = np.array(['j2','j9'], dtype=object)  # Open Health Posts
open_hcs = np.array(['j10'], dtype=object) # Open Health Centers

# Call the fitness function to evaluate the performance
satisfied_demand, lost_demand, max_dist_first_assignment, max_dist_second_assignment = fitness(
model_data, open_hps, open_hcs
)

# Output the results
print(f"Satisfied demand: {satisfied_demand}")
print(f"Lost demand: {lost_demand}")
print(f"Max distance in first assignment: {max_dist_first_assignment}")
print(f"Max distance in second assignment: {max_dist_second_assignment}")

Satisfied demand: 401.0
Lost demand: 3599.0
Max distance in first assignment: 2713.267484143831
Max distance in second assignment: 3956.219756117301
